In [7]:
rdd = sc.textFile("./supermarket.txt")

from itertools import combinations

minSupport = 0.001
minConfidence = 0.75
bcSize = rdd.count()

def countFreq(transaction):
    items = transaction.split(',')
    return [(tuple(sorted(itemsPair)), 1)
           for itemsPair in combinations(items, n)]

def generateRules(pair):
    element = pair[0]
    freq = pair[1]
    return [(element[:i] + element[i+1:], (element[i], freq)) for i in range(len(element))]

def support(pair):
    return pair[1][1] / rddSize

def confidence(pair):
    return pair[1][1] / pair[1][2]

n = 1

freqs1RDD = rdd\
            .flatMap(countFreq)\
            .reduceByKey(lambda a,b: a+b)

n = 2

freqs2RDD = rdd\
            .flatMap(countFreq)\
            .reduceByKey(lambda a,b: a+b)

order2RulesRDD = freqs2RDD.flatMap(generateRules)\
                        .join(freqs1RDD)\
                        .map(lambda x: (x[0], (x[1][0][0], x[1][0][1], x[1][1])))\
                        .filter(lambda x: support(x) > minSupport)\
                        .filter(lambda x: confidence(x) > minConfidence)\
                        .map(lambda x: f"{','.join(x[0])}  -> {x[1][0]}")

n = 3

freqs3RDD = rdd\
            .flatMap(countFreq)\
            .reduceByKey(lambda a,b: a+b)

order3RulesRDD = freqs3RDD.flatMap(generateRules)\
                        .join(freqs2RDD)\
                        .map(lambda x: (x[0], (x[1][0][0], x[1][0][1], x[1][1])))\
                        .filter(lambda x: support(x) > minSupport)\
                        .filter(lambda x: confidence(x) > minConfidence)\
                        .map(lambda x: f"{','.join(x[0])}  -> {x[1][0]}")

order2RulesRDD.union(order3RulesRDD).repartition(1).saveAsTextFile("MarketBasketRules")